In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
COLORS = [(0.12109375, 0.46484375, 0.703125),
          (0.99609375, 0.49609375, 0.0546875),
          (0.171875, 0.625, 0.171875),
          (0.8359375, 0.15234375, 0.15625),
          (0.578125, 0.40234375, 0.73828125),
          (0.546875, 0.3359375, 0.29296875),
          (0.88671875, 0.46484375, 0.7578125),
          (0.49609375, 0.49609375, 0.49609375),
          (0.734375, 0.73828125, 0.1328125),
          (0.08984375, 0.7421875, 0.80859375)]

import calendar

# Set font sizes
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20
                
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
df = pd.read_csv("../data/Elexon/UK_data.csv", index_col=0, parse_dates=True)

CARBON_INTENSITY = {"biomass": 18, "hydro": 4, "nuclear": 16,
                    "solar": 46, "gas": 469, "wind": 12,
                    "coal": 1001, "oil": 840}
col_map = dict(zip(['Biomass', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil',
       'Hydro Pumped Storage', 'Hydro Run-of-river and poundage',
       'Nuclear', 'Other', 'Solar', 'Wind Offshore', 'Wind Onshore'],
                  ['biomass', "gas", "coal", "oil", "hydrop", "hydror", "nuclear", "other", "solar", "wind1", "wind2"]))

df = df.rename(col_map, axis=1)
df["wind"] = df["wind1"] + df["wind2"]
df["hydro"] = df["hydrop"] + df["hydror"]
df = df.drop(["wind1", "wind2", "hydrop", "hydror", "other"], axis=1)
df = df.fillna(0)

df["total"] = df.sum(axis=1)
print("Dropping %d rows with 0 generation" % len(df[df["total"] == 0.]))
df = df[df["total"] > 0.]
df["carbon_intensity"] = df.apply(
    lambda row:sum(row[fuel] * CARBON_INTENSITY[fuel] for fuel
                   in CARBON_INTENSITY)/row["total"], axis=1)

df["year"] = df.index.year
df["month"] = df.index.month
df["hour"] = df.index.hour

# Compute totals for dispatchable generation
cols = ["biomass", "hydro", "nuclear", "gas", "oil", "coal"]
df["total_D"] = df[cols].sum(axis=1)
df["carbon_D"] = df.apply(lambda row:sum(row[fuel]*CARBON_INTENSITY[fuel]
                                           for fuel in cols)/1e3, axis=1)

In [ ]:
aefs = df.loc[:, ["year", "hour", "carbon_intensity"]].groupby(["year", "hour"]).mean()
aefs = aefs.unstack(level='year')
aefs.columns = aefs.columns.droplevel()

mefs = dict()
df_diff = df.diff().dropna()
df_diff["hour"] = df_diff.index.hour
for y in range(2015, 2019):
    mefs[y] = []
    start = pd.to_datetime("%d-01-01" %y)
    end = pd.to_datetime("%d-01-01" % (y+1))
    for h in range(24):
        sel = (df_diff.hour == h) & (df_diff.index>start) & (df_diff.index<end)

        lr = LinearRegression()
        lr.fit(df_diff[sel].total_D.values.reshape(-1,1), df_diff[sel].carbon_D.values.reshape(-1,1))
        mefs[y].append(1000 * lr.coef_[0][0])
mefs = pd.DataFrame(mefs)

In [ ]:
f, ax = plt.subplots()

ax.axvspan(0, 6, facecolor='b', alpha=0.05)
ax.axvspan(19, 23, facecolor='b', alpha=0.05)
ax.axvspan(6, 19, facecolor='y', alpha=0.05)
ax.text(10, 100, "DAYTIME")

for i, y in enumerate(range(2015, 2019)):
    ax.plot(mefs[y], label=str(y), color=COLORS[i])
    ax.plot(aefs[y], color=COLORS[i], marker='o')

ax.grid(True)
ax.set_xlim([0,23])
ax.set_ylim([0,800])
ax.set_xlabel('Hour of the day')
ax.set_title('(b) Great Britain: hourly AEFs and MEFs');
ax.set_ylabel('kg/MWh');
ax.set_ylabel('kg/MWh');
ax.set_yticks([0, 150, 300, 600, 750])
ax.set_yticks([CARBON_INTENSITY["solar"], CARBON_INTENSITY["gas"]], minor=True)
ax.set_yticklabels(["Solar", "Gas"], minor=True)
plt.savefig('figures/fig1b.pdf', bbox_inches='tight')
plt.savefig('figures/fig1b.png', bbox_inches='tight')

# Save data for later use
mefs.to_csv('figures/UK_mefs.csv')
aefs.to_csv('figures/UK_aefs.csv')

In [ ]:
f, ax = plt.subplots(figsize=(15, 5))
ax.plot(df.carbon_intensity)
ax.set_ylabel("AEFs (kg/MWh)")
ax.grid()
plt.savefig('figures/UK_AEFs.pdf', bbox_inches='tight')

In [ ]:
src_cols = ['biomass', 'gas', 'coal', 'oil', 'nuclear',
            'solar', 'wind', 'hydro']
df_plot = df.loc[:, ["year", "month", "total"] + src_cols].groupby(["year", "month"]).sum()
df_plot = df_plot.reset_index()
df_plot.index = pd.to_datetime((df_plot.year*10000+df_plot.month*100+1).apply(str),format='%Y%m%d')
f, ax = plt.subplots(figsize=(15, 5))
for src in src_cols:
    ax.plot(100*(df_plot[src]/df_plot.total), label=src)
ax.set_ylabel("Grid mix (%)")
ax.legend()
ax.grid()
plt.savefig('figures/UK_gridmix.pdf', bbox_inches='tight')

In [ ]:
df_ann = df.loc[:, ["year", "total"] + src_cols].groupby(["year"]).sum()
df_ann

In [ ]:
print("\t\t2015\t2016\t2018")
for col in src_cols + ["total"]:
    data15 = 100 * df_ann.loc[2015, col] / df_ann.loc[2015, "total"]
    data16 = 100 * df_ann.loc[2016, col] / df_ann.loc[2016, "total"]
    data18 = 100 * df_ann.loc[2018, col] / df_ann.loc[2018, "total"]
    print("%s\t:\t%.2f\t%.2f\t%.2f" % (col, data15, data16, data18))